In [1]:
%run PSO.ipynb #da qui possiamousare la funzione PSO

Best position found:  [-0.00509341 -0.01087979  0.00124978] with an evaluation of:  0.00014587460610553132


In [2]:
import keras
import numpy as np
from keras.datasets import reuters
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import to_categorical, plot_model
from keras.preprocessing.text import Tokenizer
from keras.optimizers import SGD, Adam, RMSprop
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,Flatten
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)


In [4]:
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

8982 train sequences
2246 test sequences


In [5]:
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

46 classes


In [6]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=10000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Vectorizing sequence data...
x_train shape: (8982, 10000)
x_test shape: (2246, 10000)


In [7]:
print('Convert class vector to binary class matrix for use with categorical_crossentropy')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix for use with categorical_crossentropy
y_train shape: (8982, 46)
y_test shape: (2246, 46)


In [13]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

def MLP(particle_position):
    '''This function returns the accuracy of a neural network model with the parameter set as the component of the particle position.
     The first component of the position will be the batch size, the second willl be the optimizer and the third will be the learning rate. 
     particle_position[0]: batch_size
     particle_position[1]: optimizer (<1: SGD; within [1,2): SGD with Momentum; within [2,3): Adam; within [3,inf]: RMSPROP )
     particle_position[2]: learning_rate'''
    
    batch_s = particle_position[0]
    batch_s = int(batch_s)
    optimiz = particle_position[1]
    lr = particle_position[2]
    
    model = Sequential()
    model.add(Dense(512, input_shape=(10000,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    early_stopping = EarlyStopping(monitor='val_loss',patience=3, verbose=1, mode='min', baseline=None)
    callbacks = [early_stopping]

    if optimiz < 1:
        #use SGD
        opt = SGD(learning_rate = lr)
        
    elif optimiz >=1 and optimiz < 2:
        #use SGD with Momentum
        opt = SGD(momentum = 0.9, learning_rate = lr)

    elif optimiz >= 2 and optimiz < 3:
        #use ADAM
        opt = Adam(learning_rate = lr)

    else:
        #use RMSPROP
        opt = RMSprop(learning_rate = lr)

    model.compile(optimizer= opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=batch_s,
                    validation_split=0.2,
                    callbacks=callbacks)
    
    last_accuracy = history.history['val_accuracy'][-1]

    return last_accuracy

In [9]:
'''plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()'''

"plt.plot(history.history['loss'], label='Training loss')\nplt.plot(history.history['val_loss'], label='Validation loss')\nplt.title('Training and validation loss')\nplt.xlabel('Epochs')\nplt.ylabel('Loss')\nplt.legend()\nplt.show()"

In [10]:
'''plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()'''

"plt.plot(history.history['accuracy'], label='Training accuracy')\nplt.plot(history.history['val_accuracy'], label='Validation accuracy')\nplt.title('Training and validation accuracy')\nplt.xlabel('Epochs')\nplt.ylabel('Accuracy')\nplt.legend()\nplt.show()"

In [14]:
best_parameter, best_accuracy = PSO(swarm_size = 10,
                                    dim = 3,
                                    evaluation_funct = MLP,
                                    lower_bound=[0,0,0.0001],
                                    upper_bound = [128,4,0.01],
                                    v_max = [64,2,0.001],
                                    problem = 'maximum',
                                    max_iteration = 2,
                                    termination_criteria = ['fixed_iteration', 'swarm_radius'])

print('Best position found: ', best_parameter, 'with an accuracy of: ',best_accuracy)

Epoch 1/20
102/102 [==============================] - 8s 71ms/step - loss: 1.2991 - accuracy: 0.7208 - val_loss: 0.9321 - val_accuracy: 0.8013
Epoch 2/20
102/102 [==============================] - 7s 73ms/step - loss: 0.6453 - accuracy: 0.8539 - val_loss: 0.9221 - val_accuracy: 0.8114
Epoch 3/20
102/102 [==============================] - 9s 84ms/step - loss: 0.4839 - accuracy: 0.9017 - val_loss: 1.0831 - val_accuracy: 0.8019
Epoch 4/20
102/102 [==============================] - 7s 71ms/step - loss: 0.4437 - accuracy: 0.9198 - val_loss: 1.2538 - val_accuracy: 0.7986
Epoch 5/20
102/102 [==============================] - 8s 76ms/step - loss: 0.4959 - accuracy: 0.9205 - val_loss: 1.2267 - val_accuracy: 0.7974
Epoch 5: early stopping
Epoch 1/20
103/103 [==============================] - 9s 76ms/step - loss: 1.2756 - accuracy: 0.7271 - val_loss: 0.9151 - val_accuracy: 0.8013
Epoch 2/20
103/103 [==============================] - 7s 71ms/step - loss: 0.5863 - accuracy: 0.8717 - val_loss: 0.899

TypeError: cannot unpack non-iterable NoneType object